In [1]:
# imports
import gzip
import math
import random
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import string
from scipy.sparse import csr_matrix
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [2]:
# converts sentences to tokens
punct = string.punctuation
def word2token(sent):
        t = sent.lower() # lowercase string
        t = "".join([c for c in t if not (c in punct)]) # non-punct characters
        words = word_tokenize(t)# tokenizes
        return words

In [3]:
# reading in first 500,001 entries of DS
file = 'beer.json.gz'
def readData(path):
    x = gzip.open(path)
    for l in x:
        yield eval(l)

In [5]:
count = 0
d_train = []
d_test = []
amount_in_train = 400000
for d in readData(file):
    if count > 500000:
        break
    if count < amount_in_train:
        d_train.append(d)
    if count > amount_in_train:
        d_test.append(d)
    count = count + 1

In [6]:
user_rating = defaultdict(list)
beer_rating = defaultdict(list)
brew_rating = defaultdict(list)
for d in d_train:
    beer_rating[d['beer/beerId']].append(d['review/overall'])
    user_rating[d['user/profileName']].append([d['beer/beerId'] , d['review/overall']])
    brew_rating[d['beer/brewerId']].append( d['review/overall'])

In [8]:
beer_avg = defaultdict(int)
for b in beer_rating:
    beer_avg[b] = np.mean(beer_rating[b])


In [9]:
user_avg = defaultdict(list)
for u in user_rating:
    for b in user_rating[u]:
        user_avg[u].append(b[1] - beer_avg[b[0]])

In [10]:
real_u_avg = defaultdict(int)
for u in user_avg:
    real_u_avg[u] = np.mean(user_avg[u])


In [11]:
avg_beer = []
for d in d_train:
    avg_beer.append(d['review/overall'])
bavg = np.mean(avg_beer)

In [12]:
# does brewery sell better beer
brew_avg = defaultdict(int)
for br in brew_rating:
    brew_avg[br] = np.mean(brew_rating[br]) - bavg

In [13]:
train_text = []
train_user = []
train_rating = []
train_brew = []
for d in d_train:
    train_brew.append(d['beer/brewerId'])
    train_text.append(d['review/text'])
    train_user.append(d['user/profileName'])
    train_rating.append(d['review/overall'])

In [14]:
test_text = []
test_user = []
test_rating = []
test_brew = []
for d in d_test:
    test_brew.append(d['beer/brewerId'])
    test_text.append(d['review/text'])
    test_user.append(d['user/profileName'])
    test_rating.append(d['review/overall'])

In [25]:
pipeline = Pipeline([
    ('Tf-Idf', TfidfVectorizer(ngram_range=(1,2), analyzer=word2token)),
    ('regressor', linear_model.Ridge())
])

In [26]:
pipeline.fit(train_text, train_rating)

Pipeline(steps=[('Tf-Idf',
                 TfidfVectorizer(analyzer=<function word2token at 0x7f42d9435d30>,
                                 ngram_range=(1, 2))),
                ('regressor', Ridge())])

In [27]:
pip_pred = pipeline.predict(test_text)

In [28]:
pip_pred = [5 if p>5 else p for p in pip_pred]

In [29]:
mean_absolute_error(test_rating, pip_pred)

0.3990474385484594

In [30]:
new_preds = []
for i in range(len(pip_pred)):
    user = real_u_avg[test_user[i]]/4
    brew = brew_avg[test_brew[i]]/4
    new_preds.append(pip_pred[i]+user+brew)

In [31]:
mean_absolute_error(test_rating, new_preds)

0.39654424584614373